In [46]:
# Importing Libraries
from tensorflow import keras
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras import optimizers, losses
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing import image

import pickle
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [47]:
# Base Path for all files
data_dir = '/kaggle/input/lumpy-skin-images-dataset/Lumpy Skin Images Dataset'

In [48]:
###### Using ImageDataGenerator to load the Images for Training and Testing the CNN Model
datagenerator = {
    "train": ImageDataGenerator(horizontal_flip=True,
                                vertical_flip=True,
                                rescale=1. / 255,
                                validation_split=0.1,
                                shear_range=0.1,
                                zoom_range=0.1,
                                width_shift_range=0.1,
                                height_shift_range=0.1,
                                rotation_range=30,
                               ).flow_from_directory(directory=data_dir,
                                                     target_size=(256, 256),
                                                     subset='training',
                                                    ),

    "valid": ImageDataGenerator(rescale=1 / 255,
                                validation_split=0.1,
                               ).flow_from_directory(directory=data_dir,
                                                     target_size=(256, 256),
                                                     subset='validation',
                                                    ),
}

Found 922 images belonging to 2 classes.
Found 102 images belonging to 2 classes.


1. Data Preprocessing:

**Image Data Generator:**

Define an ImageDataGenerator object to augment your training data. This helps the model generalize better and avoid overfitting. Use techniques like random flips, rotations, and scaling to create variations of your images.

In [49]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
validation_datagen = ImageDataGenerator(rescale=1./255,
                                         validation_split=0.1) 


**Data Flow:**

Use the generators with flow_from_directory to create data generators that will automatically load and preprocess your images based on the class labels in the folders.

In [50]:
train_generator = train_datagen.flow_from_directory(
    data_dir,  # Base directory
    target_size=(224, 224),  # Resize images to 224x224 (common for DenseNet)
    batch_size=32,  # Train in batches of 32
    class_mode='binary'  # Binary classification (Lumpy Skin or Normal)
)

validation_generator = validation_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'  # Use validation split from directory
)


Found 1024 images belonging to 2 classes.
Found 102 images belonging to 2 classes.


2. Model Building:

**Base Model (Transfer Learning):**
Define your base model using DenseNet121. Since you already have pre-trained weights for DenseNet, this is a good choice. Set include_top=False to exclude the final classification layers.

In [51]:
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


**Freezing Base Model Layers (Optional):**

Consider freezing the base model layers to prevent them from being updated during training. This can be helpful if you have a limited dataset to avoid overfitting the pre-trained weights.

In [52]:
base_model.trainable = False  # Freeze base model layers


**Adding Classification Layers:**
1. Create a Sequential model and add the pre-trained base model (without the top layers).
1. Add additional convolutional layers for further feature extraction if needed.
1. Use GlobalAveragePooling2D for efficient feature extraction.
1. Add Dense layers with dropout for regularization. Finally, add a Dense layer with 1 unit and sigmoid activation for binary classification (Lumpy Skin or Normal).

In [53]:
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(128, activation='relu'))  # Additional dense layer with ReLU activation
model.add(Dropout(0.5))  # Dropout for regularization
model.add(Dense(1, activation='sigmoid'))  # Output layer with sigmoid activation

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


3. Training the Model:

**Model Training:**

Use the model.fit function to train the model with your training data generator (train_generator) and validation data generator (validation_generator). Specify the number of epochs (iterations over the data) and steps per epoch

In [54]:
model.fit(train_generator,
          epochs=10,  # Adjust based on dataset size and validation performance
          validation_data=validation_generator)


Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 140s 4s/step - accuracy: 0.6599 - loss: 0.7407 - val_accuracy: 0.8922 - val_loss: 0.2925
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 117s 4s/step - accuracy: 0.8470 - loss: 0.3693 - val_accuracy: 0.8922 - val_loss: 0.2366
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 142s 4s/step - accuracy: 0.9001 - loss: 0.2667 - val_accuracy: 0.9216 - val_loss: 0.1941
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 141s 4s/step - accuracy: 0.8681 - loss: 0.3130 - val_accuracy: 0.9118 - val_loss: 0.2002
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 141s 3s/step - accuracy: 0.8952 - loss: 0.2548 - val_accuracy: 0.9510 - val_loss: 0.1566
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 142s 3s/step - accuracy: 0.9015 - loss: 0.2581 - val_accuracy: 0.9216 - val_loss: 0.1872
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 115s 3s/step - accuracy: 0.8982 - loss: 0.2567 - val_accuracy: 0.9314 - val_loss: 0.1583
Epoch 8/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 142s 3s/step - accuracy: 0.9046 - loss: 0.2401 - val_accuracy: 0.8922 - v

**Empty Data Generators:**



In [55]:
for sample in train_generator:
    images, labels = sample
    # Print the shape of images and labels to confirm data
    print(images.shape, labels.shape)
    break  # Only print a few samples

for sample in validation_generator:
    images, labels = sample
    # Print the shape of images and labels to confirm data
    print(images.shape, labels.shape)
    break  # Only print a few samples


(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)


In [56]:
import os
print(os.listdir(data_dir))  # List contents of your data directory


['Normal Skin', 'Lumpy Skin']


In [61]:
# Viewing the summary of the model
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ densenet121 (Functional)        │ (None, 7, 7, 1024)     │     7,037,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 1024)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │       131,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,431,493 (28.35 MB)

 Trainable params: 131,329 (513.00 KB)

 Non-trainable params: 7,037,504 (26.85 MB)

 Optimizer params: 262,660 (1.00 MB)

4. Evaluation and Saving
**Evaluate the Model:**

Use the **model.evaluate** function to evaluate the model's performance on the validation data.

In [57]:
model.evaluate

<bound method TensorFlowTrainer.evaluate of <Sequential name=sequential_2, built=True>>

**Save the Model:**

Use model.save('lumpy_skin_classifier.h5') to save the trained model for future use

In [60]:
model.save('/kaggle/working/lumpy_skin_classifier123.h5')

ValueError: Unable to synchronously create dataset (name already exists)